In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/MyDrive/AILAB/final_dataset.csv')
print(df.shape)
df.head()

(2669, 6)


,id,text,Garge Archana Atul,Archana Kumari,Priyanshu Raj,Final
0,1,Sassy...? More like femme bots than killer blo...,1,1.0,1,1.0
1,2,I've had better looking shits than these two! ...,1,1.0,2,1.0
2,3,The girls can cook for me anytime. Just not ho...,1,1.0,2,1.0
3,4,The face of very ugly promo girls ! Faces like...,1,1.0,1,1.0
4,5,@mykitchenrules Elegant and beautiful?Cheap an...,1,1.0,1,1.0


In [5]:
df=df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)
print(df.isna().sum())
y=df['Final']

id                    0
text                  0
Garge Archana Atul    0
Archana Kumari        0
Priyanshu Raj         0
Final                 0
dtype: int64


In [6]:
%%time
import re

text=' '.join(df['text'])
text=text.split()
freq_comm = pd.Series(text).value_counts()
rare=freq_comm[freq_comm.values==1]
corpus=[]

def get_clean_text(x):
  if type(x) is str:
    x=x.lower()
    x=re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x)
    x=re.sub('rt', '', x)
    x=re.sub('[^A-Z a-z]+','',x)
    corpus.append(x)
    x=' '.join([t for t in x.split() if t not in rare])
    return x
  else:
    return x
df['text']=df['text'].apply(lambda x: get_clean_text(x))

CPU times: user 96.7 ms, sys: 1.33 ms, total: 98 ms
Wall time: 99.1 ms


In [7]:
lemma_female=['girl', 'woman', 'women', 'female']
lemma_male=['boy', 'man', 'men', 'male']
necessity_qualifiers=['must', 'should', 'ought', 'required', 'have to', 'has to']
quantity_qualifiers=['all', 'only', 'most']
plural_form=['women', 'men', 'girls', 'boys', 'females', 'males']


In [56]:
from pandas.core.arrays.numeric import T
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")


biased_adj={}
biased_verb={}
def verb_biased(text):
  bv=[]
  ba=[]
  doc = nlp(text)
  for token in doc:
    if token.lemma_ in lemma_female or token.lemma_ in lemma_male:
      if token.pos_ =='VERB':
        bv.append(token)
      elif token.pos_ =='ADJ':
        ba.append(token)
  
  if len(ba)>0:
    biased_adj[text]=ba
  
  if len(bv)>0:
    biased_verb[text]=bv


In [9]:
print(corpus)

['sassy more like femme bots than killer blondes mkr', 'ive had better looking shits than these two mkr mkr killerblondes', 'the girls can cook for me anytime just not hot pot or spatchcock mkr', 'the face of very ugly promo girls  faces like cats arsehole mkr excited to see themgo down tonightliterally httptcohgojrfoieo', 'mykitchenrules elegant and beautifulcheap and trashynothing more unattractive than girls banging on about how hot hey are mkr notsassy', 'obviously natural blondes mkr', 'stop saying dumb blondes with pretty faces as you need a pretty face to pull that off mkr', 'newideamagazinethey werent kidding when they said deconstructed mkr yeh seen better shit in womens magazi oh rightsorry mkr', 'mkr omg there restaurant looks like a a little girls room oh my gosh', 'yes we get it youre pretty tone down the self promo and just cook mykitchenrules mkr', 'mkr oh my gawd we are like so hot i cant even httptcoyiskqvhmkh', 'drasko only likes his mums hot pot especially when shes 

In [57]:
for j in corpus:
  verb_biased(j)

In [58]:
for key in biased_verb:
  print(key, " : ", biased_verb[key])

comattwow chubssays again talking about feminists not you man  :  [man]
 popeharrigan and were lost i told you this didnt take us anywhere fuuck you female drivers no sexist bro  :  [female]
 cwoho some woman was proper staring at me whilst i was parking my car its ok babe i male therefore i can drive notsexist truth  :  [male]
fuck off julescarbonaro and call me sexist but females tryna b presidentjust stop theres a reason why its called the founding fathers   :  [females]
 isaiahblood peyton manning need to change his name to peyton girling because thats how hes throwing nooffenseladies notsexist  :  [manning]
 thewoolerite im not sexist but fuck womens half pipe is weak  :  [womens]
 noberger womans half pipe is hard to watch im not sexist but this is brutal one finished run in the last  athletes iwanttocry  :  [womans]
 shaunmichael im not sexistbut man womens basketball on espn is unwatchable  :  [womens]
 tgud am not sexist but most woman cant drive no offence girls your the best

In [59]:
for key in biased_adj:
  print(key, " : ", biased_adj[key])

i want to see the butchered girls go up against the texan who has male sass and his daughter who is actually good looking mkr  :  [male]
mykitchenrules steve drasko is the male equivalent of those faux models wannabe mkr  :  [male]
honestly some of the rougher female heads on tv are compiled right here mkr  :  [female]
thegags  nope  jessi and many other feminists love our male tears though httptcozruzpba  :  [male]
 ryanklang i need feminism because male happiness oppresses me fight manpiness not to be confused with manpenis fight that  twe  :  [male]
 pmb gamergate obviously madden is a sexist video game series that refuses to add female characters tweetlikeafeminist  :  [female]
pndiamond i saw him but i rarely engage male femszero point to it  they are just following orders  :  [male]
 olewahr mt  self made man norah vincent chooses female privilege over male privilege httpstcoymwxwaqgv  :  [female, male]
hayleyjoness why do you judge on genitals rather than merit  if a man works i

In [60]:
print(len(biased_verb))
print(len(biased_adj))

14
169
